In [1]:
%pip install -U -q google-cloud-aiplatform langchain-core langchain-google-vertexai langchain-text-splitters langchain-experimental "unstructured[all-docs]" pypdf pydantic lxml pillow matplotlib opencv-python tiktoken


Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""

from fastapi import APIRouter, Depends
from fastapi.exceptions import HTTPException
from vertexai.generative_models import GenerativeModel, Content
from common.utils.auth import verify_oauth2_token
from models.chat_session import ChatSession

import base64
import os
import uuid
import re

from typing import List, Tuple
from IPython.display import display, Image, Markdown
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_text_splitters import CharacterTextSplitter

from langchain_google_vertexai import (
    VertexAI,
    ChatVertexAI,
    VertexAIEmbeddings,
    VectorSearchVectorStore,
)

from unstructured.partition.pdf import partition_pdf

from google.cloud import aiplatform

PROJECT_ID = os.getenv("PROJECT_ID")
GCP_REGION = os.getenv("GCP_REGION")
GCS_BUCKET_URI = os.getenv("GCS_BUCKET_URI")

aiplatform.init(project=PROJECT_ID,
                location=GCP_REGION,
                staging_bucket=GCS_BUCKET_URI)


In [3]:
# Extract images, tables, and chunk text from a PDF file.
raw_pdf_elements = partition_pdf(
  filename="./sample_docs/Introducing Gemini 1.5, Google's next-generation AI model.pdf",
  extract_images_in_pdf=False,
  infer_table_structure=True,
  chunking_strategy="by_title",
  max_characters=4000,
  new_after_n_chars=3800,
  combine_text_under_n_chars=2000,
  image_output_dir_path="data/"
)

print(raw_pdf_elements)


This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?